출처: https://mmdetection3d.readthedocs.io/en/latest/user_guides/config.html

<핵심>
- dict()에서 type은 클래스 혹은 함수 이름이며,
- 학습 시에 RUNNER가 dict를 참조하여 Registry에 등록된 모듈을 임포트한다.

#### LEARN ABOUT CONFIGS
MMDetection3D와 다른 OpenMMLab 레포지토리는 MMEngine’s config system을 사용합니다. MMEngine의 config 시스템은 모듈식 설계를 사용하며 상속 디자인을 바탕으로 되어 있습니다. 해당 구조는 다양한 실험을 수행하는데 편리합니다.

### Config file content
MMDetection3D는 모듈식 설계를 사용하며, Config 파일을 통해 서로 다른 기능을 가진 모듈을 구성할 수 있다. 아래 예제에서는 PointPillars를 예로 들어, 다양한 기능 모듈을 가지는 각 필드를 소개한다.


### Model Config
model 은 디택션 알고리즘 구성 요소를 설정하는데 사용한다. 그 외에도 아래의 모듈을 구성하기 위한 설정을 포함한다.  
- type
- data_preprocessor
- voxel_encoder
- middle_encoder
- backbone
- neck
- bbox_head
- train_cfg
- test_cfg

In [3]:
model = dict(
    type='VoxelNet',
    data_preprocessor=dict(          # data_processor: dataloader가 출력한 데이터 일괄 처리
        voxel=True,
        voxel_layer=dict(
            max_num_points=32,
            point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
            voxel_size=[0.16, 0.16, 4],
            max_voxels=(16000, 40000)
        )
    ),
    voxel_encoder=dict(
        type='PillarFeatureNet',
        in_channels=4,
        feat_channels=[64],
        with_distance=False,
        voxel_size=[0.16, 0.16, 4],
        point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1]
    ),
    middel_encoder=dict(
        type='PointPillarScatter',
        in_channels=64,
        output_shape=[496, 432]
    ),
    backbone=dict(
        type='SECOND',
        in_channels=64,
        layer_nums=[3,5,5],
        layer_strides=[2,2,2],
        out_channles=[64, 128, 256]
    ),
    neck=dict(
        type='SECONDFPN',
        in_channels=[64, 128, 256],
        upsample_strides=[1, 2, 4],
        out_channels=[128, 128, 128]        
    ),
    bbox_head=dict(
        type='Anbchor3DHead',
        num_classes=3,
        in_channels=384,
        feat_channels=384,
        use_direction_classifier=True,
        assign_per_class=True,
        anchor_generation=dict(
            type='AlignedAnchor3DRangeGenerator',
            ranges=[[0, -39.68, -0.6, 69.12, 39.68, -0.6],
                    [0, -39.68, -0.6, 69.12, 39.68, -0.6],
                    [0, -39.68, -1.78, 69.12, 39.68, -1.78]],
            sizes=[[0.8, 0.6, 1.73], [1.76, 0.6, 1.73], [3.9, 1.6, 1.56]],
            rotations=[0, 1.57],
            reshape_out=False
        ),
        diff_rad_by_sin=True,
        bbox_coder=dict(type='DeltaXYZWLHRBBoxCoder'),
        loss_cls=dict(
            type='mmdet.FocalLoss',
            use_sigmoid=True,
            gamma=2.0,
            alpha=0.25,
            loss_weight=1.0            
        ),
        loss_bbox=dict(
            type='mmdet.SmoothL1Loss',
            beta=0.1111111111111111,
            loss_weight=2.0
        ),
        loss_dir=dict(
            type='mmdet.CrossEntrophyLoss', 
            use_sigmoid=False,
            loss_weight=2.0
        )
    ),
    train_cfg=dict(   # train_cfg, test_cfg는 컴포넌트의 훈련/테스트 하이퍼 파라미터이다.
        assigner=[
            dict(
                type='Max3DIoUAssigner',
                iou_calculator=dict(type='BboxOverlapsNearest3D'),
                pos_iou_thr=0.5,
                neg_iou_thr=0.35,
                min_pos_iou=0.35,
                ignore_iof_thr=-1
            ),
            dict(
                type='Max3DIoUAssigner',
                iou_calculator=dict(type='BboxOverlapsNearest3D'),
                pos_iou_thr=0.5,
                neg_iou_thr=0.35,
                min_pos_iou=0.35,
                ignore_iof_thr=-1
            ),
            dict(
                type='Max3DIoUAssigner',
                iou_calculator=dict(type='BboxOverlapsNearest3D'),
                pos_iou_thr=0.6,
                neg_iou_thr=0.45,
                min_pos_iou=0.45,
                ignore_iof_thr=-1
            )
        ],
        allowed_border=0,
        pos_weight=1,
        debug=False
    ),
    test_cfg=dict(
        use_rotate_nms=True,
        nms_across_levels=False,
        nms_thr=0.01,
        score_thr=0.1,
        min_bbox_size=0,
        nms_pre=100,
        max_num=50
    )
)

### Dataset and evaluator config
데이터 로더는 Training, Validation, Runner Test을 위해 사용된다. 데이터 로더를 빌드하기 위해 데이터 셋 과 데이터 파이프라인을 설정해야 한다. 해당 부분은 복잡하기 때문에, 중간 변수를 사용하여 데이터로더 구성 작성을 단순화 한다.

In [5]:
dataset_type = 'KittiDataset'
data_root = 'data/kitti/'
class_names = ['Pedestrian', 'Cyclist', 'Car']
point_cloud_range = [0, -39.68, -3, 69.12, 39.68, 1]
input_modality = dict(use_lidar=True, use_camera=False)
metainfo = dict(classes=class_names)

db_sampler = dict(
    data_root = data_root,
    info_path = data_root + 'kitti_dbinfos_train.pkl',
    rate=1.0,
    prepare=dict(
        filter_by_difficulty=[-1],
        filter_by_min_points=dict(Car=5, Pedestrian=5, Cyclist=5)
    ),
    classed=class_names,
    sample_groups=dict(Car=15, Pedestrian=15, Cyclist=15),
    points_loader=dict(
        type='LoadPointsFromFile',
        coor_type='LIDAR',
        load_dim=4,
        use_dim=4
    )
)

In [ ]:
train_pipeline = [
    dict(type='LoadPointsFromFile', coord_type='LIDAR', load_dim=4, use_dim=4),
    dict(type='LoadAnnotations3D', with_bbox_3d=True, with_label_3d=True),
    dict(type='ObjectSample', db_sampler=db_sampler, use_ground_plane=True),
    dict(type='RandomFlip3D', flip_ratio_bev_horizontal=0.5),
    dict(
        type='FlobalRotScaleTrans',
        rot_range=[-0.78539816, 0.78539816],
        scale_ratio_range=[0.95, 1.05]
    ),
    dict(type='PointsRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='PointShuffle'),
    dict(
        type='Pack3DDetInputs',
        keys=['points', 'gt_labels_3d', 'gt_bboxes_3d']
    )
]

test_pipeline = [
    dict(type='LoadPointsFromFile', coord_type='LIDAR', load_dim=4, use_dim=4),
    dict(
        type='MultiScaleFlipAug3D',
        img_scale=(1333, 800),
        pts_scale_ratio=1,
        flip=False,
        transforms=[
            dict(
                type='GlobalRotScaleTrans',
                rot_range=[0, 0],
                scale_ratio_range=[1., 1.],
                translation_std=[0, 0, 0]),
            dict(type='RandomFlip3D'),
            dict(type='PointsRangeFilter', point_cloud_range=point_cloud_range)
        ]),
    dict(type='Pack3DDetInputs', keys=['points'])
]

eval_pipeline = [
    dict(type='LoadPointsFromFile', coord_type='LIDAR', load_dim=4, use_dim=4),
    dict(type='Pack3DDetInputs', keys=['points'])
]

In [ ]:
train_dataloader = dict(
    batch_size=6,
    num_workers=4,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    dataset = dict(
        type='RepeatDataset',
        times=2,
        dataset=dict(
            type=dataset_type,
            data_root=data_root,
            ann_file='kitti_infos_train.pkl',
            data_predix=dict(pts='traininig/velodyne_reduced'),        
            pipeline=train_pipeline,
            modality=input_modality,
            test_mode=False,
            metainfo=metainfo,
            box_type_3d='LiDAR')
        )
)

val_dataloader = dict(
    batch_size=1,
    num_workers=1,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        data_prefix=dict(pts='training/velodyne_reduced'),
        ann_file='kitti_infos_val.pkl',
        pipeline=test_pipeline,
        modality=input_modality,
        test_mode=True,
        metainfo=metainfo,
        box_type_3d='LiDAR')
)

test_dataloader = dict(
    batch_size=1,
    num_workers=1,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        data_prefix=dict(pts='training/velodyne_reduced'),
        ann_file='kitti_infos_val.pkl',
        pipeline=test_pipeline,
        modality=input_modality,
        test_mode=True,
        metainfo=metainfo,
        box_type_3d='LiDAR')
)

[Evaluators](https://mmengine.readthedocs.io/en/latest/tutorials/evaluation.html)는 학습된 모델의 메트릭을 계산하기 위해 사용된다. 



In [ ]:
val_evaluator = dict(
    type='KittiMetric',
    ann_file=data_root + 'kitti_infos_val.pkl',
    metric='bbox'
)
test_evaluator = val_evaluator

test 데이터 셋은 어노테이션 파일이 없으므로, 일반적으로 test_evaluator config와 val_evaluator config는 일반적으로 동일하다.
만약, 디택션 결과를 테스트 데이터 셋에 한해 저장하고 싶다면 아래와 같이 config 파일을 구성할 수 있다.

In [ ]:
test_dataloader = dict(
    batch_size=1,
    num_workers=1,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        data_prefix=dict(pts='testing/velodyne_reduced'),
        ann_file='kitti_infos_test.pkl',
        load_eval_anns=False,
        pipeline=test_pipeline,
        modality=input_modality,
        test_mode=True,
        metainfo=metainfo,
        box_type_3d='LiDAR')
)

test_evaluator = dict(
    type='KittiMetric',
    ann_file=data_root + 'kitti_infos_test.pkl',
    metric='bbox',
    format_only=True,
    submission_prefix='results/kitti-3class/kitti_results'
)

#### Training and testing config
MMEngine's runner는 Loop를 사용하여 학습, 검증, 테스트 프로세스를 제어한다. 사용자는 최대 Epoch 횟수 및 검증 간격을 정할 수 있다.

In [ ]:
train_cfg = dict(
    type='EpochBasedTrainLoop',
    max_epochs=80,
    val_interval=2
)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')

#### Optimization config
[optim_wrapper](https://mmengine.readthedocs.io/en/latest/tutorials/optim_wrapper.html)는 최적화 설정 필드이다. 최적화 설정 기능 뿐만 아니라, gradient clipping, mixed precision training 등의 훈련도 지원한다.

In [ ]:
# Optimizer wrapper config
optim_wrapper = dict(  
    # Optimizer wrapper type, switch to AmpOptimWrapper 
    # to enable mixed precision training.
    type='OptimWrapper',  
    # Optimizer config. Support all kinds of optimizers in PyTorch. 
    # Refer to https://pytorch.org/docs/stable/optim.html#algorithms
    optimizer=dict(  
        type='AdamW', 
        lr=0.001, 
        betas=(0.95, 0.99), 
        weight_decay=0.01),
    # Gradient clip option. 
    # Set None to disable gradient clip. 
    # Find usage in https://mmengine.readthedocs.io/en/latest/tutorials/optim_wrapper.html
    clip_grad=dict(max_norm=35, norm_type=2))  

#### Param_scheduler
[param_scheduler](https://mmengine.readthedocs.io/en/latest/tutorials/param_scheduler.html)는 학습률, 및 모맨텀 과 같은 하이퍼 파라미터 조정하는 방법 구성 필드이다.

In [ ]:
param_scheduler = [
    dict(
        type='CosineAnnealingLR',
        T_max=32,
        eta_min=0.01,
        begin=0,
        end=32,
        by_epoch=True,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=48,
        eta_min=1.0000000000000001e-07,
        begin=32,
        end=80,
        by_epoch=True,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingMomentum',
        T_max=32,
        eta_min=0.8947368421052632,
        begin=0,
        end=32,
        by_epoch=True,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingMomentum',
        T_max=48,
        eta_min=1,
        begin=32,
        end=80,
        by_epoch=True,
        convert_to_iter_based=True),
]            

#### Hook config
사용자는 훈련, 검증, 테스트 루프에 hook을 추가하여 일부 동작을 수행할 수 있습니다
두개의 다른 hook 필드가 존재하고 default_hooks, custom_hooks가 그 종류이다.  
default_hook은 런타임에 필요한 hook 구성이다. 기본 우선순위를 가지며, 사용자는 None으로 동작을 disable 할 수 있다.

In [ ]:
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=50),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=-1),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='Det3DVisualizationHook'))

custom_hooks = []

## Runtime config

In [ ]:
# The default registry scope to find modules. 
# Refer to https://mmengine.readthedocs.io/en/latest/advanced_tutorials/registry.html
default_scope = 'mmdet3d'  

env_cfg = dict(
    cudnn_benchmark=False,  # Whether to enable cudnn benchmark
    mp_cfg=dict(  # Multi-processing config
        # Use fork to start multi-processing threads. 
        # 'fork' usually faster than 'spawn' but maybe unsafe. 
        # See discussion in https://github.com/pytorch/pytorch/issues/1355
        mp_start_method='fork', 
        # Disable opencv multi-threads to avoid system being overloaded 
        opencv_num_threads=0),  
    dist_cfg=dict(backend='nccl'))  # Distribution configs

# Visualization backends. 
# Refer to https://mmengine.readthedocs.io/en/latest/advanced_tutorials/visualization.html
vis_backends = [dict(type='LocalVisBackend')]  
visualizer = dict(
    type='Det3DLocalVisualizer', 
    vis_backends=vis_backends, name='visualizer'
)

# Whether to format logs with epoch type. 
# Should be consistent with the train loop's type.
log_processor = dict(
    type='LogProcessor',  # Log processor to process runtime logs
    window_size=50,  # Smooth interval of log values
    by_epoch=True
)  

# The level of logging.
log_level = 'INFO'
# Load model checkpoint as a pre-trained model from a given path. 
# This will not resume training.  
load_from = None
# Whether to resume from the checkpoint defined in `load_from`. 
# If `load_from` is None, it will resume the latest checkpoint in the `work_dir`.  
resume = False  

#### Config file inheritance

configs/_base, dataset, model, schedule, default_runtime 아래에는 4가지 기본 구성 요소가 있다.  
SECOND, PointPillars, PartA2, VoteNet 과 같은 모델 중 하나를 사용하여 많은 방법을 쉽게 구성할 수 있다.  
_base_ 구성요소로 구성된 cofigs를 primitive 라고 한다.

동일한 폴더 아래 모든 구성에 대해 하나의 primitive 구성한 가지는 것이 권장된다.
다른 구성은 _base_ 구성에서 상속 받도록 한다. 이러한 방식으로 최대 상속 레벨은 '3' 이다.

쉽게 이해하면, 기여자자가 기존 메소드에서 상속하는 것이 추천된다. 예를 들어, PointPillars를 기반으로 수정을 한다면, 사용자는 _base_ = '../pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py' 를 상속받아야 하면, config에서 필요한 필드를 수정한다.

기존 메소드와 구조를 공유하지 않는 새로운 메소드를 구성한다면, xxx_rcnn 폴더를 configs 아래에 만들 수 있다.


In [ ]:
_base_ = './pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py'

# _base_가 여러 파일인 경우, 여러 파일에서 상속하는 것을 의미한다.
_base_ = [
    '../_base_/models/pointpillars_hv_secfpn_kitti.py',
    '../_base_/datasets/kitti-3d-3class.py',
    '../_base_/schedules/cyclic-40e.py', '../_base_/default_runtime.py'
]


#### Ignore some fields in the base configs
경우에 따라 _delete=True로 config 설정을 무시하도록 할 수 있다.

In [ ]:
model = dict(
    type='MVXFasterRCNN',
    data_preprocessor=dict(voxel_layer=dict(...)),
    pts_voxel_encoder=dict(...),
    pts_middle_encoder=dict(...),
    pts_backbone=dict(...),
    
    pts_neck=dict(
        type='FPN',
        norm_cfg=dict(type='naiveSyncBN2d', eps=1e-3, momentum=0.01),
        act_cfg=dict(type='ReLU'),
        in_channels=[64, 128, 256],
        out_channels=256,
        start_level=0,
        num_outs=3),
        
    pts_bbox_head=dict(...))

FPN 및 SECONDFPN 다른 키워드를 사용하여 다음을 구성한다.

In [ ]:
_base_ = '../_base_/models/pointpillars_hv_fpn_nus.py'
model = dict(
    
    pts_neck=dict(
        _delete_=True, # <--------pts_neck을 다음 config 로 대체 한다.
        type='SECONDFPN',
        norm_cfg=dict(type='naiveSyncBN2d', eps=1e-3, momentum=0.01),
        in_channels=[64, 128, 256],
        upsample_strides=[1, 2, 4],
        out_channels=[128, 128, 128]),

    pts_bbox_head=dict(...)
)    

#### Use intermediate variables in configs
일부 중간 변수는 데이터 세트의 train_pipeline/test_pipeline 과 같은 configs 파일 안에서 사용된다.
중간 변수를 자식 config에서 수정해야 한다면, 사용자는 중간 변수를 상응하는 필드에 다시 집어넣어야 하므로 무용지물이라고 할 수 있다.  
예를 들어, multi-scale 전략을 PointPillar 학습, 테스트에서 사용하고자 할때, train_pipeline/test_pipeline 은 수정하고자 하는 중간변수 일 수 있다.  
처음에 먼저 train_pipeline/test_pipeline을 정의하고, dataloader 필드로 넘긴다.

In [ ]:
_base_ = './nus-3d.py'
train_pipeline = [
    dict(
        type='LoadPointsFromFile',
        load_dim=5,
        use_dim=5,
        backend_args=backend_args),
    dict(
        type='LoadPointsFromMultiSweeps',
        sweeps_num=10,
        backend_args=backend_args),
    dict(type='LoadAnnotations3D', 
        with_bbox_3d=True,
        with_label_3d=True),

    dict(
        type='GlobalRotScaleTrans',
        rot_range=[-0.3925, 0.3925],
        scale_ratio_range=[0.95, 1.05],
        translation_std=[0, 0, 0]),

    dict(type='RandomFlip3D', flip_ratio_bev_horizontal=0.5),
    dict(type='PointsRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectNameFilter', classes=class_names),
    dict(type='PointShuffle'),

    dict(
        type='Pack3DDetInputs',
        keys=['points', 'gt_labels_3d', 'gt_bboxes_3d'])
]

test_pipeline = [
    dict(
        type='LoadPointsFromFile',
        load_dim=5,
        use_dim=5,
        backend_args=backend_args),
    dict(
        type='LoadPointsFromMultiSweeps',
        sweeps_num=10,
        backend_args=backend_args),
    dict(
        type='MultiScaleFlipAug3D',
        img_scale=(1333, 800),
        pts_scale_ratio=[0.95, 1.0, 1.05],
        flip=False,
        transforms=[
            dict(
                type='GlobalRotScaleTrans',
                rot_range=[0, 0],
                scale_ratio_range=[1., 1.],
                translation_std=[0, 0, 0]),
            dict(type='RandomFlip3D'),
            dict(
                type='PointsRangeFilter', 
                point_cloud_range=point_cloud_range)
        ]),
    dict(
        type='Pack3DDetInputs',
        keys=['points'])
]
train_dataloader = dict(dataset=dict(pipeline=train_pipeline))
val_dataloader = dict(dataset=dict(pipeline=test_pipeline))
test_dataloader = dict(dataset=dict(pipeline=test_pipeline))

#### Reuse variables in _base_ file 
- _base_ 파일에서 변수 재사용
- 사용자가 기본 파일의 변수를 다시 사용하려는 경우, 해당 변수의 복사본을 가져올 수 있다.


In [ ]:
_base_ = './pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class.py'

a = {{_base_.model}}  # variable `a` is equal to the `model` defined in `_base_`